# Programme data

This notebook prepares data for the Programme theme page.

In [1]:
import json
from datetime import date

from utils.paths import SITE
from utils.themes.programme_slice import ProgrammeSlice

In [2]:
EVENTS = SITE / 'themes/programme/_data/events'
EVENTS.mkdir(exist_ok=True, parents=True)

## Events data

Read events

In [3]:
programme_data = ProgrammeSlice(range=(date.min, date.today()))

Create an aggregate by month of the events

In [4]:
(
    programme_data.events
    .aggregate('month', {
        'Events': ('events', sum),
        'Audience': ('audience', sum),
        'Participants': ('participants', sum),
        # 'Records': (len),
    })
    .convert('month', lambda f: f.isoformat())
    .tocsv(EVENTS / 'total_by_month.csv')
)

Aggregate by Project and by Month, and convert months to columns

In [5]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project.csv')
)

In [6]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'audience')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project_audience.csv')
)

In [7]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'participants')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project_participants.csv')
)

Aggregate by Project and by Month, and convert projects to columns

In [8]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='month', variablefield='project_name', missing=0)
    .tocsv(EVENTS / 'monthly_breakdown.csv')
)

## Project summaries

Create a project breakdown

In [9]:
programme_data.project_breakdown.selecteq('evaluation_category', 'Digital')

project_name,evaluation_category,audience,events,manual_events,manual_participants_community,participants,projected_events,schedule_events,schedule_participants_community
Bradford 2025 Website,Digital,1427381,0,None,None,0,None,None,None
Bradford Glow,Digital,0,0,0,None,0,None,None,None
Bradford On Foot,Digital,0,0,0,2162,2162,None,None,None
Bradford Progress,Digital,0,0,0,None,0,None,None,None
CHANNELS,Digital,0,0,0,786,786,None,None,None


In [10]:
with open(EVENTS / 'by_project.json', 'w') as f:
    json.dump(
        dict(
            programme_data.project_details.records()
        ),
        f,
        indent=2,
    )

Create a summary file

In [11]:
programme_data.excluded_events_data

row,project_id,project_name,programme_category,evaluation_category,month,venue,source,date,start_date,end_date,variable,value,validation
3993,,Bradford Glow,[],Digital,2025-12-01,None,Manual,2025-12-31,2025-12-31,2025-12-31,manual_events,0,after_requested_date_range
28,rec0PhMaC6JQ0VaDk,Wandering Imaginations: Emerging Fantasy Writer Exchange,"['Literature', 'International', 'Exhibition']",Exhibition,2025-08-01,None,Airtable::Project Hub,2025-08-01,2025-08-01,2025-08-31,schedule_events,1,after_requested_date_range
28,rec0PhMaC6JQ0VaDk,Wandering Imaginations: Emerging Fantasy Writer Exchange,"['Literature', 'International', 'Exhibition']",Exhibition,2025-08-01,None,Airtable::Project Hub,2025-08-01,2025-08-01,2025-08-31,projected_events,31,after_requested_date_range
28,rec0PhMaC6JQ0VaDk,Wandering Imaginations: Emerging Fantasy Writer Exchange,"['Literature', 'International', 'Exhibition']",Exhibition,2025-08-01,None,Airtable::Project Hub,2025-08-01,2025-08-01,2025-08-31,audience,0,after_requested_date_range
28,rec0PhMaC6JQ0VaDk,Wandering Imaginations: Emerging Fantasy Writer Exchange,"['Literature', 'International', 'Exhibition']",Exhibition,2025-08-01,None,Airtable::Project Hub,2025-08-01,2025-08-01,2025-08-31,schedule_participants_community,0,after_requested_date_range


In [12]:
with open(EVENTS / 'summary.json', 'w') as f:
    json.dump(
        {
            'total': {
                'events': sum(programme_data.events.values('events')),
                'audience': sum(a for a in programme_data.events.values('audience') if a is not None),
                'participants': sum(filter(None.__ne__, programme_data.events.values('participants')), 0),
            },
            'excluded': dict(programme_data.excluded_events_data.aggregate('validation', len).records()),
            'date': {
                'earliest': min(programme_data.events_data.values('start_date')).isoformat(),
                'latest': max(programme_data.events_data.values('end_date')).isoformat(),
            }
        },
        f,
        indent=2,
    )

/tmp/ipykernel_3058484/735255026.py:7: DeprecationWarning: NotImplemented should not be used in a boolean context
  'participants': sum(filter(None.__ne__, programme_data.events.values('participants')), 0),


## Venues

In [13]:
from utils.themes.programme import Programme

In [14]:
education_settings = (
    Programme.venues
    .selectcontains('Org/Venue Type', 'Education Setting')
)

In [15]:
loading_bay = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Loading Bay')
    .addfield('event_report_count', lambda r: len(r['Event Reports']))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
loading_bay

Organisation &/or Venue Name,value
Loading Bay,8
Loading Bay (previously 26 piccadilly),0
Loading Bay 2nd Floor,0
Loading Bay Basement,17
Loading Bay Gallery,51


In [16]:
beacon = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Beacon - ')
    .addfield('event_report_count', lambda r: len(r['Event Reports'] or []))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
beacon

Organisation &/or Venue Name,value
Beacon - Bowling Park,59
Beacon - Cliffe Castle,0
Beacon - Lister Park,0
Beacon - Wibsey Park,56


In [17]:
json.dump(
    {
        'loading_bay': dict(loading_bay.records()),
        'beacon': dict(beacon.records()),
    },
    open(EVENTS / 'by_venue.json', 'w')
)